<a href="https://colab.research.google.com/github/IkutoYoshioka/Econometrics/blob/main/%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$【固定効果モデル】$  

\begin{equation}
Y_{it} = α_i + λ_t + β_1X1_{it} + β_2X2_{it} + \boldsymbol{\beta}\boldsymbol{X}_{it} + u_{it}
\end{equation}  
$α_i：時間で一定な都道府県ごとに異なる効果（都市ダミーやその県特有の文化・産業など）$  
$λ_t：各都道府県に同じ影響を与えるが時間によって変化する効果（政策などのマクロ的な要因）$  
$\boldsymbol{\beta}\boldsymbol{X}_{it}：コントロール変数(X_3,...,X_{11})とその係数の行列表記$  
$u_{it}：誤差項$  
$cov(α_i,X) \not= 0 の可能性が高い$   
$cov(α_i,X) = 0  ならばランダム効果モデル$

$【ランダム効果モデル】$  

\begin{equation}
Y_{it} = ν_i + β_1X1_{it} + β_2X2_{it} + \boldsymbol{\beta}\boldsymbol{X}_{it} + u_{it}
\end{equation}  
$ν_i：変量効果。α_iと同じ意味を持つが変数Xと無相関$  
$ランダム効果モデルなら時間不変な変数である都市ダミーを入れられるが固定効果に基づく推定結果の方が信頼できるらしい$   


  
* 説明変数と個体効果が相関している可能性が高い場合:  
固定効果モデル（FE）を選択。  
例: 都道府県ごとの産業構造がGDP成長率に影響を与える場合。
産業構造（説明変数）が地域の特性（観測されない固定効果）と相関している可能性が高い。  
* 説明変数と個体効果が無相関であると仮定できる場合:  
ランダム効果モデル（RE）を選択。  
例: 各国の投資額がGDP成長率に与える影響を分析する場合。
投資額（説明変数）が各国固有の特性（文化や地理）と無相関と仮定できるなら、REモデルが効率的。
* 判断に迷う場合:  
両方のモデルを推定し、Hausman検定で適切なモデルを選ぶ。  
P値 < 0.05 → 固定効果モデル（FE）。  
P値 >= 0.05 → ランダム効果モデル（RE）。

In [2]:
pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00


In [9]:
import pandas as pd
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS

# データの読み込み
file_path = "/content/drive/MyDrive/応用マクロゼミ/ゼミ資料/財政経済分析/修正メインデータ.csv"
data = pd.read_csv(file_path)

# 数値変換が必要な列の処理（例：カンマを除去して数値化）
data['X3'] = data['X3'].str.replace(',', '').astype(float)
data['X6'] = data['X6'].str.replace(',', '').astype(float)
data['X7'] = data['X7'].str.replace(',', '').astype(float)
data['X8'] = data['X8'].str.replace(',', '').astype(float)
data['X9'] = data['X9'].str.replace(',', '').astype(float)

# '調査年'を時間変数、'地域'を個体識別子として設定
data['調査年'] = data['調査年'].str.replace('年度', '').astype(int)  # 年度を数値化
data = data.set_index(['地域', '調査年'])


In [11]:
# 相関行列の計算
print(data.corr())

          X3        X4        X5        X6        X7        X8        X9  \
X3  1.000000  0.592738 -0.461322  0.660412  0.987164  0.832654  0.822586   
X4  0.592738  1.000000 -0.961530  0.298412  0.573399  0.458674  0.453926   
X5 -0.461322 -0.961530  1.000000 -0.168441 -0.437399 -0.326522 -0.318488   
X6  0.660412  0.298412 -0.168441  1.000000  0.737393  0.816716  0.843283   
X7  0.987164  0.573399 -0.437399  0.737393  1.000000  0.897255  0.882667   
X8  0.832654  0.458674 -0.326522  0.816716  0.897255  1.000000  0.979765   
X9  0.822586  0.453926 -0.318488  0.843283  0.882667  0.979765  1.000000   
X2  0.853190  0.614809 -0.510723  0.751095  0.857052  0.699264  0.696890   
Y   0.004127 -0.254021  0.244326  0.164638  0.007818  0.006003  0.039921   
X1  0.811716  0.534160 -0.398760  0.808692  0.830966  0.734097  0.737379   

          X2         Y        X1  
X3  0.853190  0.004127  0.811716  
X4  0.614809 -0.254021  0.534160  
X5 -0.510723  0.244326 -0.398760  
X6  0.751095  0.164638 

In [12]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF の計算
X = data[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']]  # 説明変数を選択
vif = pd.DataFrame()
vif['Variable'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)


  Variable         VIF
0       X1  116.690863
1       X2   84.352314
2       X3  254.219534
3       X4  108.943591
4       X5   82.327672
5       X6  325.748889
6       X7  323.716269
7       X8   54.526220
8       X9   46.920979


> VIFが10以上で多重共線性が強い。せめてX3,X6,X7をモデルから外す。

### 固定効果モデル

In [14]:
from linearmodels.panel import compare

# 説明変数の一部を削除してモデルを検証
model1 = PanelOLS.from_formula('Y ~ X1 + X2 + EntityEffects + TimeEffects', data=data)
model2 = PanelOLS.from_formula('Y ~ X1 + X2 + X9 + EntityEffects', data=data)
model3 = PanelOLS.from_formula('Y ~ X1 + X2 + X9 + EntityEffects + TimeEffects', data=data)
model4 = PanelOLS.from_formula('Y ~ X1 + X2 + X4 + X8 + X9 + EntityEffects + TimeEffects', data=data)
model5 = PanelOLS.from_formula('Y ~ X1 + X2 + X4 + X5 + X8 + X9 + EntityEffects + TimeEffects', data=data)
model6 = PanelOLS.from_formula('Y ~ X1 + X2 + X4 + X5 + X8 + X9', data=data)
model7 = PanelOLS.from_formula('Y ~ X1 + X2 + X4 + X5 + EntityEffects + TimeEffects', data=data)


results1 = model1.fit(cov_type='clustered', cluster_entity=True)
results2 = model2.fit(cov_type='clustered', cluster_entity=True)
results3 = model3.fit(cov_type='clustered', cluster_entity=True)
results4 = model4.fit(cov_type='clustered', cluster_entity=True)
results5 = model5.fit(cov_type='clustered', cluster_entity=True)
results6 = model6.fit(cov_type='clustered', cluster_entity=True)
results7 = model7.fit(cov_type='clustered', cluster_entity=True)

# モデルの比較
print(compare({'Model 1': results1, 'Model 2': results2, 'Model 3': results3, 'Model 4': results4, 'Model 5': results5, 'Model 6': results6, 'Model 7': results7}))


                                                      Model Comparison                                                      
                              Model 1       Model 2       Model 3        Model 4        Model 5        Model 6       Model 7
----------------------------------------------------------------------------------------------------------------------------
Dep. Variable                       Y             Y             Y              Y              Y              Y             Y
Estimator                    PanelOLS      PanelOLS      PanelOLS       PanelOLS       PanelOLS       PanelOLS      PanelOLS
No. Observations                  564           564           564            564            564            564           564
Cov. Est.                   Clustered     Clustered     Clustered      Clustered      Clustered      Clustered     Clustered
R-squared                      0.0269        0.1572        0.0426         0.0479         0.0479         0.1383        0.0285


### ランダム効果モデルと固定効果モデル、Hausman検定

In [21]:
from linearmodels.panel import PanelOLS, RandomEffects, compare

# ランダム効果モデルを作成
random_model1 = RandomEffects.from_formula('Y ~ X1 + X2', data=data)
random_model2 = RandomEffects.from_formula('Y ~ X1 + X2 + X9', data=data)
random_model3 = RandomEffects.from_formula('Y ~ X1 + X2 + X9', data=data)
random_model4 = RandomEffects.from_formula('Y ~ X1 + X2 + X4 + X8 + X9', data=data)
random_model5 = RandomEffects.from_formula('Y ~ X1 + X2 + X4 + X5 + X8 + X9', data=data)
random_model6 = RandomEffects.from_formula('Y ~ X1 + X2 + X4 + X5 + X8 + X9', data=data)
random_model7 = RandomEffects.from_formula('Y ~ X1 + X2 + X4 + X5', data=data)

# 各モデルの結果を計算
fixed_results1 = model1.fit(cov_type='clustered', cluster_entity=True)
random_results1 = random_model1.fit()

fixed_results2 = model2.fit(cov_type='clustered', cluster_entity=True)
random_results2 = random_model2.fit()

fixed_results3 = model3.fit(cov_type='clustered', cluster_entity=True)
random_results3 = random_model3.fit()

fixed_results4 = model4.fit(cov_type='clustered', cluster_entity=True)
random_results4 = random_model4.fit()

fixed_results5 = model5.fit(cov_type='clustered', cluster_entity=True)
random_results5 = random_model5.fit()

fixed_results6 = model6.fit(cov_type='clustered', cluster_entity=True)
random_results6 = random_model6.fit()

fixed_results7 = model7.fit(cov_type='clustered', cluster_entity=True)
random_results7 = random_model7.fit()

# Hausman検定を比較する
comparison1 = compare({"Fixed Effects": fixed_results1, "Random Effects": random_results1})
comparison2 = compare({"Fixed Effects": fixed_results2, "Random Effects": random_results2})
comparison3 = compare({"Fixed Effects": fixed_results3, "Random Effects": random_results3})
comparison4 = compare({"Fixed Effects": fixed_results4, "Random Effects": random_results4})
comparison5 = compare({"Fixed Effects": fixed_results5, "Random Effects": random_results5})
comparison6 = compare({"Fixed Effects": fixed_results6, "Random Effects": random_results6})
comparison7 = compare({"Fixed Effects": fixed_results7, "Random Effects": random_results7})

# 結果を表示

print("Model 1:\n", comparison1)
print("Model 2:\n", comparison2)
print("Model 3:\n", comparison3)
print("Model 4:\n", comparison4)
print("Model 5:\n", comparison5)
print("Model 6:\n", comparison6)
print("Model 7:\n", comparison7)


Model 1:
                     Model Comparison                   
                        Fixed Effects    Random Effects
-------------------------------------------------------
Dep. Variable                       Y                 Y
Estimator                    PanelOLS     RandomEffects
No. Observations                  564               564
Cov. Est.                   Clustered        Unadjusted
R-squared                      0.0269            0.0296
R-Squared (Within)             0.0528            0.0332
R-Squared (Between)           -11.165           -0.0753
R-Squared (Overall)           -0.3230            0.0296
F-statistic                    6.9776            8.5647
P-value (F-stat)               0.0010            0.0002
=====================     ===========   ===============
X1                             0.1074            0.0496
                             (1.8486)          (3.5094)
X2                            -5.8697           -3.9101
                            (-1.1899) 

In [17]:
import numpy as np
from scipy.stats import chi2
import pandas as pd

# Hausman検定を行う関数
def hausman_test(beta_fe, var_fe, beta_re, var_re):
    beta_diff = beta_fe - beta_re
    var_diff = var_fe - var_re

    # 分散行列の逆行列を計算
    try:
        var_diff_inv = np.linalg.inv(var_diff)
        H = beta_diff.T @ var_diff_inv @ beta_diff
        p_value = 1 - chi2.cdf(H, len(beta_fe))
        return H, p_value
    except np.linalg.LinAlgError:
        return np.nan, np.nan

# 各モデルの固定効果とランダム効果のデータ
models = {
    "Model 1": {
        "beta_fe": np.array([0.1074, -5.8697]),
        "se_fe": np.array([1.8486, 1.1899]),
        "beta_re": np.array([0.0496, -3.9101]),
        "se_re": np.array([3.5094, 3.0499])
    },
    "Model 2": {
        "beta_fe": np.array([0.0930, -9.7628, 2.081e-06]),
        "se_fe": np.array([1.7645, 2.1244, 2.5371]),
        "beta_re": np.array([0.0555, -4.8243, 6.36e-08]),
        "se_re": np.array([3.8191, 3.4672, 1.6758])
    },
    "Model 3": {
        "beta_fe": np.array([0.0909, -2.7074, 6.658e-07]),
        "se_fe": np.array([1.5037, 0.6957, 2.3272]),
        "beta_re": np.array([0.0555, -4.8243, 6.36e-08]),
        "se_re": np.array([3.8191, 3.4672, 1.6758])
    },
    "Model 4": {
        "beta_fe": np.array([0.0834, -0.8319, 0.2965, -1.053e-06, 7.646e-07]),
        "se_fe": np.array([1.3544, 0.1919, 1.7034, 4.2302, 2.7555]),
        "beta_re": np.array([0.1309, -5.8069, -0.0503, -2.298e-06, 5.555e-07]),
        "se_re": np.array([5.4034, 4.1049, 4.3601, 4.0374, 3.6616])
    },
    "Model 5": {
        "beta_fe": np.array([0.0833, -0.8043, 0.2639, -0.0349, -1.065e-06, 7.679e-07]),
        "se_fe": np.array([1.3337, 0.1831, 0.5678, 0.0658, 3.4166, 2.6038]),
        "beta_re": np.array([0.0854, -1.9406, -0.1204, 0.1676, -2.128e-06, 5.475e-07]),
        "se_re": np.array([3.4386, 1.2692, 7.2939, 5.7869, 3.8404, 3.7117])
    },
    "Model 6": {
        "beta_fe": np.array([0.0854, -1.9406, -0.1204, 0.1676, -2.128e-06, 5.475e-07]),
        "se_fe": np.array([2.6667, 1.2855, 2.8122, 3.3097, 2.8470, 2.5109]),
        "beta_re": np.array([0.0854, -1.9406, -0.1204, 0.1676, -2.128e-06, 5.475e-07]),
        "se_re": np.array([3.4386, 1.2692, 7.2939, 5.7869, 3.8404, 3.7117])
    },
    "Model 7": {
        "beta_fe": np.array([0.1019, -5.5801, 0.5031, 0.3699]),
        "se_fe": np.array([1.7431, 1.1597, 1.1867, 0.7402]),
        "beta_re": np.array([0.0871, -2.2749, -0.1189, 0.1729]),
        "se_re": np.array([3.6935, 1.4797, 7.3767, 5.9726])
    }
}

# Hausman検定結果を格納するリスト
results = []

# 各モデルについてHausman検定を実行
for model_name, params in models.items():
    beta_fe = params["beta_fe"]
    var_fe = np.diag(params["se_fe"] ** 2)

    beta_re = params["beta_re"]
    var_re = np.diag(params["se_re"] ** 2)

    H, p_value = hausman_test(beta_fe, var_fe, beta_re, var_re)

    results.append({
        "Model": model_name,
        "Hausman Statistic": H,
        "P-value": p_value
    })

# 結果をデータフレームに変換して表示
results_df = pd.DataFrame(results)
results_df


,Model,Hausman Statistic,P-value
0,Model 1,-0.487317,1.0
1,Model 2,-3.248322,1.0
2,Model 3,-0.388511,1.0
3,Model 4,-1.479628,1.0
4,Model 5,-0.822596,1.0
5,Model 6,0.000000,1.0
6,Model 7,-12.942646,1.0


> ？

> model6を最適なモデルと判断

In [18]:
print(random_results6)

                        RandomEffects Estimation Summary                        
Dep. Variable:                      Y   R-squared:                        0.1383
Estimator:              RandomEffects   R-squared (Between):             -1.9875
No. Observations:                 564   R-squared (Within):               0.2120
Date:                Sat, Jan 25 2025   R-squared (Overall):              0.1383
Time:                        05:08:01   Log-likelihood                   -1375.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      14.929
Entities:                          47   P-value                           0.0000
Avg Obs:                       12.000   Distribution:                   F(6,558)
Min Obs:                       12.000                                           
Max Obs:                       12.000   F-statistic (robust):             14.929
                            